# Barotropic Instability

We will represent an incompressible, inviscid, homogeneous Boussinesq fluid to simulate a Barotropic Instability.

This exercise was designed for the course Waves and Instabilities in Geophysical Fluid Dynamics of the Master's Degree in Advanced Physics and Applied Mathematics, at University of the Balearic Islands (Spain).

Author: Daniel Argüeso
Email: d.argueso@uib.es

Feb-2023

## Import modules

In [ ]:
import numpy as np
from dedalus import public as de
from dedalus.extras import flow_tools
import matplotlib.pyplot as plt
import h5py
import time

## Import and set logging

In [ ]:
import logging
root = logging.root
for h in root.handlers:
    h.setLevel("INFO")
logger = logging.getLogger(__name__)

## Define the problem

### Set the problem domain

You need to define the domain of the problem. The first two items indicate the aspect ratio. For exmaple (2,1). The second one indicates the number of grid points in each direction. They should be consistent with the aspect ratio

In [ ]:
Lx, Ly = 
nx, ny = 

### Create bases and domain

They basically define the transformation between the grid space and the spectral space. There are various types of basis, but the most popular are:
- Fourier: to define periodic functions in an intervarl (usually the direction of the flow)
- Chebyshev: general functions in an interval (they require boundary conditions, usually top and bottom)

For each basis we specify the direction, the dimensions, the interval and the dealising. Dealising is used to evaluate operators in the Fourier space and for numerical stability. We use the default 3/2.

Then, the domain, which combines both bases and the dimensions above to define the problem domain

In [ ]:
x_basis = de.Fourier("x", nx, interval=(0, Lx), dealias=3 / 2)
y_basis = de.Fourier("y", ny, interval=(0, Ly), dealias=3 / 2)
domain = de.Domain([x_basis, y_basis], grid_dtype=np.float64)

### Set parameters

In [ ]:
Beta = 15


### Define the problem and the equations


We define the problem. We have different options, but we will use Initial value problem (IVP) in all of our exercises.
When we define the problem, we have to specify the domain, the variables, the parameters and the equations.

In the first command, we take the domain specifications we defined before and we define the variables that the problem will use. For the barotropic instability, we will need a single variable, the streamfunction.

Because the boundary conditions will be set through the vorticity, we will also need to define vorcitity as a function of the streamfunction, but the problem definition will only need one variable and one equation. To define new variables as a function of others, you can use:
problem.substitutions["y"] = "f(x)" 

In [ ]:
#define the problem 

Next, we define the parameters in the problem's framework, so that they can be used in the equations

In [ ]:
problem.parameters['Beta'] = Beta

Now we move on to solving the problem. We need have a crear idea of what the equations of the problem will be, and then write them in dedalus notation.

We will derive the equation for the barotropic instbaility using vorticity and streamfunction, and assuming a B-plane approximattion.

$$ \frac{D}{Dt}[\omega] = 0 $$
$$ \zeta = \nabla \times \bold{u} $$
where:

$$ \omega = \zeta + f $$

and ω is the absolute vorticity, ζ is the local or relative vorticity:

$$ \zeta = \nabla \times \bold{u} $$

and f is the planetary vorticity.

If we want to study an incompressible two-dimensional flow:

$$ \bold{u} = (u,v) $$

$$ \psi = \nabla \times (\psi \hat{e}_z) $$

Thus the local vorticity meets:

$$ \zeta = \nabla^2 \psi $$

Since ∂/∂t[f] = 0, the first equation can be written as (using the beta-plane approximation):

$$ \frac{D}{Dt}ζ + u \cdot \boldsymbol{\nabla} f = 0 \rightarrow \frac{D}{Dt}ζ = -v\Beta $$ 


If we write this in terms of the stream function, we obtain the equation for the Barotropic instability:

$$ \frac{D}{Dt} [\nabla^2 {\psi}] =  -\beta \frac{\partial\psi}{\partial x}$$

Or alternatively:


$$ \partial_t \zeta =  -\beta \frac{\partial\psi}{\partial x} + \mathbb{J}(\psi,\zeta)$$



Vorticity & velocity are no longer states of the system. They are true diagnostic variables.
But you still might want to set initial condisitons based on vorticity (for example).
To do this you'll have to solve for the streamfunction.

In [ ]:
#Substitutions
problem.substitutions["v"] = "  dx(psi) "

# Add substituion for zeta as a function of psi

## Add substitution for Jacobian

In [ ]:
## Now add the equation for zeta


### Define the boundary conditions

We need only one boundary condition (equivalent to the pressure gauge)




In [ ]:
problem.add_equation("psi = 0", condition="(nx == 0) and (ny == 0)")

Because we want to solve the equation for the streamfunction, but it is more convenient to give initial conditions for vorticity, we will calculate the initial streamfunction for a given vorticity

In [ ]:
init = de.LBVP(domain, variables=["init_psi"])


Create array with noise for initial conditions

In [ ]:
gshape = domain.dist.grid_layout.global_shape(scales=1)
slices = domain.dist.grid_layout.slices(scales=1)
rand = np.random.RandomState(seed=42)
noise = rand.standard_normal(gshape)[slices]

Set the initial conditions vorticity: we need a central band of positive vorticity, and two bands of negative vorticity at the top and the bottom to create regions of zero vorticity and constant flow in between.

In [ ]:
zeta0 = domain.new_field()
zeta0.set_scales(1)
x, y = domain.grids(scales=1)

In [ ]:
zeta0["g"] = noise / 40
#Then create positive and negative vorticity horiztonal stripes
init.parameters["zeta0"] = zeta0

In [ ]:
#Complete with the relationship between zeta and psi to calculate init_psi from zeta0
init.add_equation(
        ,
        condition="(nx != 0) or  (ny != 0)",
    )
init.add_equation(" init_psi = 0", condition="(nx == 0) and (ny == 0)") #This is the boundary condition

Set initial timestep and CFL conditions

We set the initial timestep, which will be later modified to ensure stability and optimize the simulation depending on the problem itself. We can also define cfl conditions based on velocities.

In [ ]:
dt = 1e-3  # Lx/nx
    # You can set parameters to limit the size of the timestep.
    CFL = flow_tools.CFL(
        solver,
        initial_dt=dt,
        cadence=10,
        safety=2,
        max_change=1.5,
        min_change=0.5,
        max_dt=10 * dt,
    )
    CFL.add_velocities(("u", "v"))

## Solving

In this step run the solver. At the same time we save some information for analysis we may want to make later on. And draw the plots every certain number of timesteps. Saving the analysis is not necessary, but it may be helpful to modify plots without having to run the entire simulation again.

Prepare the variables that will be saved for analysis (this is optional)

In [ ]:
analysis = solver.evaluator.add_file_handler("analysis", iter=50, max_writes=1000)
# ADD analysis tasks (get vorticity as the laplacian of the streamfunction and velocities)

### Plotting initial state

### Move into solving loop

In [ ]:
#Add logger, 
#Initialize timestep,
# Initialize solver with while loop
# Create plot every N iterations

### ending program with information

In [ ]:
end_time = time.time()
logger.info('Run time: %f' %(end_time-start_time))
logger.info('Iterations: %i' %solver.iteration)